<div class="alert alert-block alert-info">
    <b>Import Packages</b>
</div>


In [2]:
import pandas as  pd
import os
import numpy as np
import dtale
import json
from tqdm import tqdm
import sys

<div class="alert alert-block alert-info">
    <b>Jupyter Plumbing</b>
</div>


In [3]:
# %load C:\users\derek\.jupyter\startup.py
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#pd.set_option('display.max_colwidth', 100)

pd.set_option('display.max_colwidth', None)



pd.set_option("display.max_rows", None)

pd.options.display.max_columns = 500

pd.options.display.max_seq_items = 500



<div class="alert alert-block alert-info">
    <b>View .csv files in our working directory</b>
</div>


In [4]:
csvStore = os.path.join(os.getcwd(),'__csvStore' )
csvlist = []
for file in enumerate(os.listdir(csvStore)):
    if file[1].endswith(".csv"):
        csvlist.append(os.path.join(str(csvStore), file[1]))
        print(os.path.join(str(csvStore), file[1]))

/home/derek/Jupyter_Notebooks/__csvStore/df_visits.csv
/home/derek/Jupyter_Notebooks/__csvStore/df_episode.csv
/home/derek/Jupyter_Notebooks/__csvStore/final.csv
/home/derek/Jupyter_Notebooks/__csvStore/df_OASIS.csv
/home/derek/Jupyter_Notebooks/__csvStore/dead_patients_final.csv
/home/derek/Jupyter_Notebooks/__csvStore/orig_data2.csv
/home/derek/Jupyter_Notebooks/__csvStore/eliminated.csv
/home/derek/Jupyter_Notebooks/__csvStore/orig_data.csv
/home/derek/Jupyter_Notebooks/__csvStore/cleaned_data.csv
/home/derek/Jupyter_Notebooks/__csvStore/event_mapping_df.csv
/home/derek/Jupyter_Notebooks/__csvStore/soc2.csv
/home/derek/Jupyter_Notebooks/__csvStore/du.csv
/home/derek/Jupyter_Notebooks/__csvStore/patients.csv
/home/derek/Jupyter_Notebooks/__csvStore/duplicate_ceo_ids.csv
/home/derek/Jupyter_Notebooks/__csvStore/soc.csv
/home/derek/Jupyter_Notebooks/__csvStore/alive_patients_final.csv
/home/derek/Jupyter_Notebooks/__csvStore/dv.csv
/home/derek/Jupyter_Notebooks/__csvStore/deduped_compl

In [5]:
list(enumerate(csvlist))

[(0, '/home/derek/Jupyter_Notebooks/__csvStore/df_visits.csv'),
 (1, '/home/derek/Jupyter_Notebooks/__csvStore/df_episode.csv'),
 (2, '/home/derek/Jupyter_Notebooks/__csvStore/final.csv'),
 (3, '/home/derek/Jupyter_Notebooks/__csvStore/df_OASIS.csv'),
 (4, '/home/derek/Jupyter_Notebooks/__csvStore/dead_patients_final.csv'),
 (5, '/home/derek/Jupyter_Notebooks/__csvStore/orig_data2.csv'),
 (6, '/home/derek/Jupyter_Notebooks/__csvStore/eliminated.csv'),
 (7, '/home/derek/Jupyter_Notebooks/__csvStore/orig_data.csv'),
 (8, '/home/derek/Jupyter_Notebooks/__csvStore/cleaned_data.csv'),
 (9, '/home/derek/Jupyter_Notebooks/__csvStore/event_mapping_df.csv'),
 (10, '/home/derek/Jupyter_Notebooks/__csvStore/soc2.csv'),
 (11, '/home/derek/Jupyter_Notebooks/__csvStore/du.csv'),
 (12, '/home/derek/Jupyter_Notebooks/__csvStore/patients.csv'),
 (13, '/home/derek/Jupyter_Notebooks/__csvStore/duplicate_ceo_ids.csv'),
 (14, '/home/derek/Jupyter_Notebooks/__csvStore/soc.csv'),
 (15, '/home/derek/Jupyter_N

<div class="alert alert-block alert-info">
    <b>Create df by import .csv files created in previous step</b>
</div>


In [6]:
df_patients = pd.read_csv(csvlist[12])#.iloc[:1000,:]
df_episodes = pd.read_csv(csvlist[1])#.iloc[:150,:]
df_visits = pd.read_csv(csvlist[0])#.iloc[:150,:]
df_OASIS = pd.read_csv(csvlist[3])#.iloc[:150,:]
df_event_mapping = pd.read_csv(csvlist[9])

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-block alert-info">
    <b>This allows us to filter our data to only a handful of patients for testing purposes</b>
</div>


In [7]:
dead = df_patients[df_patients.mortality == 1].pa_id
f"There are {len(dead)} patients"

'There are 2903 patients'

In [8]:
df_OASIS_dead = df_OASIS[df_OASIS.pa_id.isin(dead)]
df_OASIS_dead.dtypes

pa_id                        int64
epi_id                       int64
ceo_id                       int64
M0090_INFO_COMPLETED_DT     object
itemID                       int64
value                      float64
OASIS                       object
dtype: object

<div class="alert alert-block alert-info">
    <b>Creates A Dictionary with {pa_id: row_numbers/num of rows} to see which patients have the most rows (most number of visits/data)</b>
</div>


In [9]:
grpz = df_OASIS_dead.groupby('pa_id').groups
#df_OASIS.groupby('pa_id').groups[36796]
#df_OASIS.groupby('pa_id').get_group(tup)

<div class="alert alert-block alert-info">
    <b>Find the Patient w the most Rows which are good candiates for testing</b>
</div>


In [10]:
def patient_most_rows(grpz):
    maxcount = max(len(v) for v in grpz.values())
    print(f'The max rownum is {maxcount}, and the pa_id is:')
    return [k for k, v in grpz.items() if len(v) == maxcount]


In [11]:
patient_most_rows(grpz)

The max rownum is 2492, and the pa_id is:


[266796]

In [12]:
length_dict = {k: len(v) for k, v in grpz.items()}
[k for k, v in length_dict.items() if v == max(length_dict.values())]
#length_dict

[266796]

In [13]:
import heapq
dead100 = [k for k, v in length_dict.items() if v in heapq.nlargest(225, length_dict.values())]
len(dead100)

287

<div class="alert alert-block alert-info">
    <b>We will take all the 'Alive' patients since there are only 225</b>
</div>

In [14]:
alive = df_patients[df_patients.mortality==0].pa_id
f"There are, {len(alive)}, alive patients"

'There are, 225, alive patients'

<div class="alert alert-block alert-info">
    <b>This is our testing patient subset</b>
</div>

In [15]:
pa_subset = alive.tolist() + dead100
f"For testing purposes we will use {len(set(pa_subset))} patient subset"

'For testing purposes we will use 512 patient subset'

<div class="alert alert-block alert-info">
    <b>Final Filter for the main df: we can hard code specific patients if desired</b>
</div>

In [16]:




#grpidx = (17191,44654,25607,18808,16862,17018,24556,16274,26475,27968,36781,17747,37870,18011,30512,51094,38501,36509,17716,42124,16950,353347,326385,319247,318537,307867,283257,268061,347156,325432,348164,348170,350202,335984,359871,364367,320405,306152,322487,322263,308977,280963)
grpidx = pa_subset
#dfidx = np.sort(np.concatenate([df_OASIS.groupby('pa_id').indices[x] for x in grpidx]))
#df_OASIS.loc[dfidx, :]


<div class="alert alert-block alert-info">
    <b>Repopulate the original df with test data</b>
</div>


In [17]:
df_patients = df_patients[df_patients.pa_id.isin(grpidx)]
df_episodes = df_episodes[df_episodes.pa_id.isin(grpidx)]
df_visits = df_visits[df_visits.pa_id.isin(grpidx)]
df_OASIS = df_OASIS[df_OASIS.pa_id.isin(grpidx)]

In [18]:
df_OASIS.pa_id.nunique()

512

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-danger">
    <b>View datatypes - the int64 were causing a bug when serializing to JSON</b>
</div>

In [19]:
df_patients.dtypes#.head()
df_episodes.dtypes#.head()
df_visits.dtypes#.head()
df_OASIS.dtypes#.head()
df_event_mapping.dtypes#.head()

pa_id                    int64
DispositionId            int64
pa_gender               object
year_born                int64
M0906_DC_TRAN_DTH_DT    object
mortality                int64
cedd_dateofdeath        object
yod                      int64
aod                      int64
dtype: object

pa_id                    int64
epi_id                   int64
mortality                int64
epi_StartOfEpisode      object
epi_EndOfEpisode        object
epi_DischargeDate       object
epi_AdmitType           object
dcc_desc                object
dcr_desc                object
icd_code                object
icd_code_description    object
dtype: object

pa_id                       int64
epi_id                      int64
ceo_id                      int64
M0090_INFO_COMPLETED_DT    object
M0100_ASSMT_REASON         object
dtype: object

pa_id                        int64
epi_id                       int64
ceo_id                       int64
M0090_INFO_COMPLETED_DT     object
itemID                       int64
value                      float64
OASIS                       object
dtype: object

itemID     int64
OASIS     object
dtype: object

<hr style="height:2px;border-width:0;color:black;background-color:black">

# Model Setup and Data Processing

<div class="alert alert-warning">
    <b>Import packages needed to Parallel process the Data</b>
</div>

In [20]:
import joblib
from joblib import Parallel, delayed
import inspect

<div class="alert alert-warning">
    <b>View the number of cores recognized for parallel processing</b>
</div>

In [21]:

print(joblib.cpu_count() - 4)


12


<div class="alert alert-warning">
    <b>Import pyhealth classes and functions</b>
</div>

In [22]:
from pyhealth.data.base_mimic import parallel_parse_tables
from pyhealth.utils.utility_parallel import unfold_parallel
from pyhealth.utils.utility_parallel import partition_estimators
from pyhealth.utils.utility import read_csv_to_df
from pyhealth.utils.utility import make_dirs_if_not_exists

In [23]:
inspect.getfile(parallel_parse_tables)
inspect.getfile(unfold_parallel)
inspect.getfile(partition_estimators)
inspect.getfile(read_csv_to_df)
inspect.getfile(make_dirs_if_not_exists)


'/home/derek/miniconda3/envs/pyhealth/lib/python3.6/site-packages/pyhealth/data/base_mimic.py'

'/home/derek/miniconda3/envs/pyhealth/lib/python3.6/site-packages/pyhealth/utils/utility_parallel.py'

'/home/derek/miniconda3/envs/pyhealth/lib/python3.6/site-packages/pyhealth/utils/utility_parallel.py'

'/home/derek/miniconda3/envs/pyhealth/lib/python3.6/site-packages/pyhealth/utils/utility.py'

'/home/derek/miniconda3/envs/pyhealth/lib/python3.6/site-packages/pyhealth/utils/utility.py'

<div class="alert alert-warning">
    <b>Create Output Directory to Save Files</b>
</div>

In [24]:
#from pathlib import Path
#Path("pyhealth_export").mkdir(parents=True, exist_ok=True)

<div class="alert alert-warning">
    <b>Setup the Parallel Proccesing Workflow</b>
</div>

In [25]:
save_dir = os.getcwd() + '/__exported'+ '/pyhealth_export'
n_jobz = (joblib.cpu_count() - 4)  # number of parallel jobs
duration = 21600  # time window for episode generation
selection_method = 'last'
var_list = df_event_mapping.OASIS.tolist()
patient_id_list = df_patients['pa_id'].tolist()

<div class="alert alert-danger">
    <b>Custom JSON Encoder to deal with np.int64 serialization Bug</b>
</div>

In [26]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

<div class="alert alert-warning">
    <b>Execute the Parallel Data Processing</b>
</div>

In [27]:
n_patients_list, starts, n_jobs = partition_estimators(
        len(patient_id_list), n_jobs=n_jobz)

Split tasks into 12 cores...


In [ ]:
all_results = Parallel(n_jobs=n_jobs, max_nbytes=None, verbose=True)(
delayed(parallel_parse_tables)(
     patient_df=df_patients,
     admission_df=df_episodes,
      icu_df=df_visits,
     event_df=df_OASIS,
     event_mapping_df=df_event_mapping,
     duration=duration,
     save_dir=save_dir,selection_method = selection_method, var_list = var_list, patient_id_list = patient_id_list)
 for i in range(n_jobs))

<div class="alert alert-warning">
    <b>Create JSON File With Patient-Level Data</b>
</div>

In [ ]:
patient_data_loc = os.path.join(save_dir, 'patient_data_demo.json')
all_results = list(map(list, zip(*all_results)))
valid_data_list = unfold_parallel(all_results[0], n_jobs)
valid_id_list = unfold_parallel(all_results[1], n_jobs)

patient_data_list = []
for p in valid_data_list:
        patient_data_list.append(p.data)

with open(patient_data_loc, 'w') as outfile:
        json.dump(patient_data_list, outfile, cls=NpEncoder)

<div class="alert alert-warning">
    <b>Use Jupyter Magics which sends commands via Bash</b>
</div>

##### the data must be in these folders to work correctly

In [ ]:
x_data = save_dir+'/x_data'
y_data = save_dir+'/y_data'
move = save_dir+'/*_episode.csv'

In [ ]:
move

In [ ]:
!mkdir $x_data
!mkdir $y_data

In [ ]:
save_dir

In [ ]:
!mv $save_dir*_episode.csv $x_data

In [ ]:
#!Powershell.exe -Command "Move-Item -Path * -Filter *_episode.csv -Destination .\x_data"



<div class="alert alert-danger">
    <b>Examination of the Output</b>
</div>

In [ ]:
type(all_results[0][0][0])
all_results[0][0][0]
patient_data_list

<div class="alert alert-warning">
    <b>Generate Labels for Mortality or Not</b>
</div>

In [ ]:
os.path.exists(save_dir+'/y_data')

In [ ]:
with open(patient_data_loc) as f:
    patient_data_list = json.load(f)
##########################################################
output_headers = ['episode_file', 'death_indicator']
output_df = pd.DataFrame(columns=output_headers)
ydir = save_dir+'/y_data'

for i, p_id in enumerate(tqdm(patient_data_list)):
    for adm in p_id['admission_list']:
        # csv file does not exist
        if 'episode_csv' in adm.keys():
            temp_list = [adm['episode_csv'], adm['death_indicator']]
            # append to the major episode dataframe
            temp_df = pd.DataFrame(temp_list).transpose()
            temp_df.columns = output_headers
            output_df = pd.concat([output_df, temp_df], axis=0)
# change file header to lower case

output_df.columns = output_df.columns.str.lower()
if not os.path.exists(ydir):
    os.makedirs(ydir)
output_df.to_csv(
        os.path.join(save_dir+'/y_data', 'mortality.csv'),
        index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">

In [28]:
from __future__ import division
from __future__ import print_function
from pathlib import Path
import torch
print(torch.__version__)

1.8.1


In [29]:
from pyhealth.data.expdata_generator import sequencedata as expdata_generator
from pyhealth.evaluation.evaluator import func

In [30]:
inspect.getfile(expdata_generator)
inspect.getfile(func)

'/home/derek/miniconda3/envs/pyhealth/lib/python3.6/site-packages/pyhealth/data/expdata_generator.py'

'/home/derek/miniconda3/envs/pyhealth/lib/python3.6/site-packages/pyhealth/evaluation/evaluator.py'

In [31]:
### May choose any of these models
# from pyhealth.models.sequence.dipole import Dipole as model
from pyhealth.models.sequence.lstm import LSTM as model
#from pyhealth.models.sequence.gru import GRU as GRU
# from pyhealth.models.sequence.embedgru import EmbedGRU as model
# from pyhealth.models.sequence.retain import Retain as model
# from pyhealth.models.sequence.raim import RAIM as model
# from pyhealth.models.sequence.tlstm import tLSTM as model
#from pyhealth.models.sequence.rf import RandomForest as model


<div class="alert alert-warning">
    <b>Serialize the Data into Pickles for Model Input</b>
</div>

In [ ]:
#data_dir = os.path.join(save_dir, 'model_output')

In [32]:
expdata_id = '04.16.mortality'
# set up the datasets
cur_dataset = expdata_generator(expdata_id, root_dir=save_dir)
cur_dataset.get_exp_data(sel_task='mortality', data_root=save_dir)
cur_dataset.load_exp_data()
cur_dataset.show_data()


Current ExpData_ID: 04.16.mortality --- Target for MIMIC


100%|██████████| 11280/11280 [02:50<00:00, 66.02it/s]


finished X generate
finished Y generate
finished L generate
generate finished
target Task: mortality
N of features: 89
N of labels: 1
N of TrainData: 7219
N of ValidData: 1805
N of TestData: 2256
load finished
target Task: mortality
N of features: 89
N of labels: 1
N of TrainData: 7219
N of ValidData: 1805
N of TestData: 2256
------------Train--------------
x_data ['/home/derek/Jupyter_Notebooks/__exported/pyhealth_export/x_data/44084_67032_episode.csv', '/home/derek/Jupyter_Notebooks/__exported/pyhealth_export/x_data/28481_25300_episode.csv', '/home/derek/Jupyter_Notebooks/__exported/pyhealth_export/x_data/255480_115248_episode.csv']
y_data [array([1.]), array([0.]), array([1.])]
l_data [2, 2, 2]
------------Valid--------------
x_data ['/home/derek/Jupyter_Notebooks/__exported/pyhealth_export/x_data/272369_147898_episode.csv', '/home/derek/Jupyter_Notebooks/__exported/pyhealth_export/x_data/24645_70503_episode.csv', '/home/derek/Jupyter_Notebooks/__exported/pyhealth_export/x_data/2240

<div class="alert alert-warning">
    <b>Run the Chosen Model!</b>
</div>

In [34]:
# initialize the model for training
# turn on GPU by setting use_gpu to True
expmodel_id = '04.16.mortality.LTSM.cpu'
clf = model(expmodel_id=expmodel_id, n_batchsize=64, use_gpu=False, n_epoch=10, gpu_ids='0, 1')
clf.fit(cur_dataset.train, cur_dataset.valid)

use CPU recource
current task can beed seen as binaryclass; loss func L1LossSigmoid is used for optimization


tr=>epoch=9 Valid Loss: 20.629, Train Loss: 20.261: 100%|██████████| 10/10 [07:32<00:00, 45.27s/it]


In [35]:
# load the best model for inference
clf.load_model()
clf.inference(cur_dataset.test)
pred_results = clf.get_results()
print(pred_results['hat_y'])

load predictor config file from ./experiments_records/04.16.mortality.LTSM.cpu/checkouts/predictor_config.json
load best-th epoch model
current task can beed seen as binaryclass; loss func L1LossSigmoid is used for optimization
[[0.9883357 ]
 [0.98833585]
 [0.9858065 ]
 ...
 [0.9887983 ]
 [0.9858067 ]
 [0.98580706]]


In [36]:
# evaluate the model
r = func(pred_results['hat_y'], pred_results['y'])

current data evaluate using binaryclass evaluation-type


<div class="alert alert-warning">
    <b>View Results</b>
</div>

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(pred_results['y'], pred_results['hat_y'])

In [ ]:
len(pred_results['y'])
len(pred_results['hat_y'])

In [ ]:
pred_results['y'].tolist()

In [ ]:
pred_results['hat_y'].tolist()

In [ ]:
np.sum(np.abs(pred_results['y'] - pred_results['hat_y']))

In [37]:
r


{'l1': 673.64136,
 'avg_precision_micro': 0.8235047904159052,
 'avg_precision_macro': 0.8235047904159052,
 'roc_auc_score_micro': 0.6994216167162532,
 'roc_auc_score_macro': 0.6994216167162532,
 'recall': 1.0,
 'precision': 0.7044642857142858,
 'f1_score': 0.8266107909900472}

In [ ]:
np.sum(pred_results['hat_y'] * pred_results['y'])/ np.sum(pred_results['y']) if np.sum(pred_results['y'])>0 else 0.

In [ ]:
np.sum(pred_results['hat_y'] * pred_results['y'])

In [ ]:
np.sum(pred_results['y'])

In [ ]:
pred_results['y'].tolist()

In [ ]:
y_score = clf.decision_function(X_test)

In [ ]:
 %load_ext autoreload

In [38]:
from sklearn.metrics import PrecisionRecallDisplay
display = PrecisionRecallDisplay.from_predictions(pred_results['y'], pred_results['hat_y'], name="LinearSVC")
_ = display.ax_.set_title("2-class Precision-Recall curve")

AttributeError: type object 'PrecisionRecallDisplay' has no attribute 'from_predictions'

In [41]:
%matplotlib
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
#calculate precision and recall
precision, recall, thresholds = precision_recall_curve(pred_results['y'], pred_results['hat_y'])

#create precision recall curve
fig, ax = plt.subplots()
ax.plot(recall, precision, color='purple')

#add axis labels to plot
ax.set_title('Precision-Recall Curve')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')

#display plot
plt.show()

Using matplotlib backend: TkAgg


Text(0.5, 1.0, 'Precision-Recall Curve')

Text(0, 0.5, 'Precision')

Text(0.5, 0, 'Recall')

findfont: Font family ['Heiti TC'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Heiti TC'] not found. Falling back to DejaVu Sans.


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

prec, recall, _ = precision_recall_curve(pred_results['y'], pred_results['hat_y'], pos_label=1)
pr_display = PrecisionRecallDisplay(precision=prec, recall=recall).plot()

In [ ]:
import sklearn
sklearn.metrics.precision_recall_curve(pred_results['y'], pred_results['hat_y'], pos_label=1, sample_weight=None)

In [ ]:
pred_results['hat_y'].tolist()

In [ ]:
len(pred_results['hat_y'].tolist())

In [ ]:
Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
The roc_auc_score function computes the area under the receiver operating characteristic (ROC) curve, which is also denoted by AUC
By computing the area under the roc curve, the curve information is summarized in one number.

In [ ]:
clf.inference(cur_dataset.valid)